In [ ]:
import sys
import numpy as np
import os
from tqdm.notebook import tqdm
import pandas as pd
import itertools

In [ ]:
pdidx = pd.IndexSlice

In [ ]:
PATH = "../../../"
if PATH not in sys.path:
    sys.path.append(PATH)
from optimization.algorithms.utils.callbacks import SolutionTracer
from hysys_optimization.problems.dsmr.smr_opt_problem import SMROptimizationProblem

In [ ]:
SIMULTAION_PATH = "Dual SMR Simulation.hsc"

## Helper Functions and Classes

In [ ]:

def optimize_smr(algorithm, n_individuals, random_state, exp_path, force_rerun=False, max_fes=None, do_scale=True):
    if max_fes is None:
        max_fes = np.inf

    smr_prob = SMROptimizationProblem(SIMULTAION_PATH, simulation_is_visible=True, do_scale=do_scale)
    exp_dir_name = f"data/{exp_path}"

    name = os.path.join(
        exp_dir_name, f"{algorithm.name}_rs{random_state}_n{n_individuals}{'ns' if not do_scale else ''}.npz")
    # if experiment is new create its folder
    if not os.path.exists(exp_dir_name):
        os.makedirs(exp_dir_name)

    # if this run was done before load it
    if os.path.exists(name) and not force_rerun:
        loaded = np.load(name, allow_pickle=True)
        keys = list(loaded.keys())
        history = {k: loaded[k] for k in keys}
        return history, smr_prob
    # if the run wasn't done before do it and save the results
    else:
        tracer = SolutionTracer(population_callbacks={
            "solutions": lambda population: population.solutions.copy(),
            "penalized_objs": lambda population: population.solutions_objective_values.copy(),
        })
        algorithm.compile(optimization_problem=smr_prob,
                          initializer_kwargs={"n_individuals":n_individuals}, random_state=random_state)
        algorithm.optimize(fes=max_fes, callbacks=[tracer], verbose=1)
        history = tracer.history
        # evaluate constraints and actual objective of the final poppulation and save_them
        objs, constraints, penalized_objs = smr_prob.evaluate(
            algorithm.population.solutions)
        history["constraints"] = constraints
        history["objs"] = objs
        np.savez(name, solutions=np.array(tracer.history["solutions"], dtype=np.ndarray),
            penalized_objs=np.array(tracer.history["penalized_objs"], dtype=np.ndarray), constraints=constraints, objs=objs)
        return history, smr_prob

## Start Optimizaing

In [ ]:
from optimization.algorithms.de.variants import get_ncde, get_clde
from optimization.algorithms.pso.variants import get_ferpso, get_gbest_pso, get_lbest_pso

In [ ]:
MAX_GENs = 100
n_individuals = 50

max_fes = MAX_GENs * n_individuals

In [ ]:
## General params
species_radius = 0.5
species_size = int(n_individuals / 10)  

## PSO params
phi_max = 4.1

## DE params
F = 0.9
pc = 0.1
strategy = "DE/rand/1/bin"  

algorithms = [
    get_gbest_pso(phi_max=phi_max),
    get_lbest_pso(phi_max=phi_max, neighborhood_size=3),
    get_ferpso(phi_max=phi_max),
    get_ncde(F=F, C=pc), 
    get_clde(F=F, pc=pc, strategy=strategy, clearing_radius=species_radius, clearing_capacity=species_size),
]
print([algorithm.name for algorithm in algorithms])

In [ ]:
EXP_PATH = "ACCEPTED TEST"

In [ ]:

RANDOM_SEEDS = [
    2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
    31, 37, 41, 43, 47, 53, 59, 61, 67, 71,
    73, 79, 83, 89, 97, 101, 103, 107, 109, 113,
    127, 131, 137, 139, 149, 151, 157, 163, 167, 173,
    179, 181, 191, 193, 197, 199, 211, 223, 227, 229,
    233, 239, 241, 251, 257, 263, 269, 271, 277, 281,
    283, 293, 307, 311, 313, 317, 331, 337, 347, 349,
    353, 359, 367, 373, 379, 383, 389, 397, 401, 409,
    419, 421, 431, 433, 439, 443, 449, 457, 461, 463,
    467, 479, 487, 491, 499, 503, 509, 521, 523, 541
]

runs = 10
do_scale = True
FORCE_RERUN = True

In [ ]:
from collections import defaultdict
histories = defaultdict(dict)
pbar = tqdm(itertools.product(RANDOM_SEEDS[:runs], algorithms), leave=True, total=len(algorithms) * runs)
for seed, algorithm in pbar:
    pbar.set_description(f"{algorithm.name} optimizing using seed: {seed}")
    histories[algorithm.name][seed], prob = optimize_smr(algorithm=algorithm,
                                              n_individuals=n_individuals,
                                              random_state=seed,
                                              exp_path=EXP_PATH,
                                              force_rerun=FORCE_RERUN,
                                              max_fes=max_fes,
                                              do_scale=do_scale)
    print(algorithm.name, " best penalized objective: ", histories[algorithm.name][seed]["penalized_objs"][-1].max())

## Shot Top Solutions

In [ ]:
smr_prob = SMROptimizationProblem(SIMULTAION_PATH, simulation_is_visible=True, do_scale=do_scale)

In [ ]:
best_obj_series = pd.Series()
df = pd.DataFrame(index=pd.Index(RANDOM_SEEDS[:runs], name="random_state"), columns=pd.MultiIndex.from_product([list(histories.keys()), ["N MF", "M MF", "E MF", "P MF", "ib MF", "ip MF",  "Suction P", "Discharge P", "MITA1", "MITA2", "Total Power"]], names=("algorithm", "attr")), dtype=float)

for algorithm in histories:
    for seed in histories[algorithm].keys():
        mask = np.logical_and(histories[algorithm][seed]["constraints"][:, 0] >= 2.95, histories[algorithm][seed]["constraints"][:, 0] <= 3.05)
        filtered_objs = histories[algorithm][seed]["objs"][mask]
        if len(filtered_objs) == 0:
            continue
        best_idx = np.argmax(filtered_objs)
        best_idx = np.arange(len(histories[algorithm][seed]["penalized_objs"][-1]))[mask][best_idx]
        df.loc[seed, pdidx[algorithm, "Total Power"]] = - histories[algorithm][seed]["objs"][best_idx]
        if do_scale:
            df.loc[seed, pdidx[algorithm, ["N MF", "M MF", "E MF", "P MF", "ib MF", "ip MF",  "Suction P", "Discharge P"]]] = smr_prob.inverse_scale(histories[algorithm][seed]["solutions"][-1][best_idx])
        else:
            df.loc[seed, pdidx[algorithm, ["N MF", "M MF", "E MF", "P MF", "ib MF", "ip MF",  "Suction P", "Discharge P"]]] = histories[algorithm][seed]["solutions"][-1][best_idx]

        df.loc[seed, pdidx[algorithm, "Penalized Obj"]] = -histories[algorithm][seed]["penalized_objs"][-1][best_idx]
        df.loc[seed, pdidx[algorithm, ["MITA1", "MITA2"]]] = histories[algorithm][seed]["constraints"][best_idx]
df.stack("algorithm").sort_values(by="Total Power")

In [ ]:
tmp_df = df.stack("algorithm").round(4)
best_idx = tmp_df.groupby("algorithm")["Total Power"].idxmin()
best_solutions = tmp_df.loc[best_idx].sort_values(by="Total Power").droplevel(0, 0)[["N MF", "M MF", "E MF", "P MF", "ib MF", "ip MF",  "Suction P", "Discharge P", "MITA1", "MITA2", "Total Power", "Penalized Obj"]].sort_values(by="Total Power")

best_solutions